In [1]:
cd ../src

/workspace/Script/NLP/PII/src


In [2]:
import os
import gc
import sys
import json
import time
import torch
import joblib
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
# from data.data_utils import get_offset_mapping,clean_text
from data.dataset import FeedbackDataset,CustomCollator

In [4]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModel, AutoConfig

%env TOKENIZERS_PARALLELISM = true

env: TOKENIZERS_PARALLELISM=true


In [5]:
from tqdm.auto import tqdm
import re
from difflib import SequenceMatcher

import codecs
import os
from collections import Counter
from typing import Dict, List, Tuple

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from text_unidecode import unidecode
import joblib
import torch

# ======================================================================================== #
def get_text_start_end(txt, s, search_from=0):
    txt = txt[int(search_from):]
    try:
        idx = txt.find(s)
        if idx >= 0:
            st = idx
            ed = st + len(s)
        else:
            raise ValueError('Error')
    except:
        res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
        if len(res):
            st, ed = res[0][0], res[0][1]
        else:
            m = SequenceMatcher(None, s, txt).get_opcodes()
            for tag, i1, i2, j1, j2 in m:
                if tag == 'replace':
                    s = s[:i1] + txt[j1:j2] + s[i2:]
                if tag == "delete":
                    s = s[:i1] + s[i2:]

            res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
            if len(res):
                st, ed = res[0][0], res[0][1]
            else:
                idx = txt.find(s)
                if idx >= 0:
                    st = idx
                    ed = st + len(s)
                else:
                    st, ed = 0, 0
    return st + search_from, ed + search_from
    
def get_offset_mapping(full_text, tokens):
    offset_mapping = []

    current_offset = 0
    for token in tokens:
        start, end = get_text_start_end(full_text, token, search_from=current_offset)
        offset_mapping.append((start, end))
        current_offset = end

    return offset_mapping


import spacy
from spacy import displacy
from pylab import cm, matplotlib
import os

colors = {
            'NAME_STUDENT': '#8000ff',
            'EMAIL': '#2b7ff6',
            'USERNAME': '#2adddd',
            'ID_NUM': '#80ffb4',
            'PHONE_NUM': 'd4dd80',
            'URL_PERSONAL': '#ff8042',
            'STREET_ADDRESS': '#ff0000'
         }


def visualize(full_text,offset_mapping,labels):
    
    ents = []
    for offset,lab in zip(offset_mapping,labels):
        ents.append({
                        'start': int(offset[0]), 
                         'end': int(offset[1]), 
                         'label': str(lab.split('-')[1]) #+ ' - ' + str(row['discourse_effectiveness'])
                    })

    doc2 = {
        "text": full_text,
        "ents": ents,
#         "title": "idx"
    }

    options = {"ents": list(colors.keys()), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)


In [6]:
import spacy
from spacy import displacy
from pylab import cm, matplotlib
import os

colors = {
            'NAME_STUDENT': '#8000ff',
            'EMAIL': '#2b7ff6',
            'USERNAME': '#2adddd',
            'ID_NUM': '#80ffb4',
            'PHONE_NUM': 'd4dd80',
            'URL_PERSONAL': '#ff8042',
            'STREET_ADDRESS': '#ff0000'
         }


def visualize(full_text,offset_mapping,labels):
    
    ents = []
    for offset,lab in zip(offset_mapping,labels):
        ents.append({
                        'start': int(offset[0]), 
                         'end': int(offset[1]), 
                         'label': str(lab.split('-')[-1]) #+ ' - ' + str(row['discourse_effectiveness'])
                    })

    doc2 = {
        "text": full_text,
        "ents": ents,
#         "title": "idx"
    }

    options = {"ents": list(colors.keys()), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

# Params

In [7]:
data_path = Path(r"/database/kaggle/PII/data")
os.listdir(data_path)

['train.json',
 'mpware_mixtral8x7b_v1.1-no-i-username.json',
 'pii_dataset_fixed.csv',
 'mixtral-8x7b-v1.json',
 '.~lock.lecture2.pptx#',
 'Fake_data_1850_218.json',
 'test.json',
 'archive.zip',
 'archive',
 'pii-masking-200k.csv',
 'sample_submission.csv',
 'mpware_mixtral8x7b_v1.1.json']

In [8]:
df = pd.read_json(data_path/'train.json')
df.shape

(6807, 5)

In [9]:
df.head(2)

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."


In [10]:
LABEL2TYPE = ('NAME_STUDENT','EMAIL','USERNAME','ID_NUM', 'PHONE_NUM','URL_PERSONAL','STREET_ADDRESS','O')
len(LABEL2TYPE)

8

In [11]:
for name in LABEL2TYPE[:-1]:
    df[name] = ((df['labels'].transform(lambda x:len([i for i in x if i.split('-')[-1]==name ])>0)))*1

In [12]:
df['nb_labels'] = df['labels'].transform(lambda x:len([i for i in x if i!="O" ]))

In [13]:
df['nb_labels'].value_counts() 

0     5862
2      599
4      108
1       86
3       52
6       46
8       14
5       10
12       6
10       5
11       3
9        3
15       2
14       2
21       2
7        1
23       1
18       1
17       1
26       1
34       1
22       1
Name: nb_labels, dtype: int64

In [14]:
df[list(LABEL2TYPE)[:-1]+['nb_labels']].sum()/1031

NAME_STUDENT     0.8642
EMAIL            0.0233
USERNAME         0.0048
ID_NUM           0.0320
PHONE_NUM        0.0039
URL_PERSONAL     0.0698
STREET_ADDRESS   0.0019
nb_labels        2.6566
dtype: float64

In [15]:
891+24+5+33+4+72+2

1031

In [16]:
1/7

0.14285714285714285

In [739]:
30/7

4.285714285714286

In [14]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [15]:
seeds = [42]
folds_names = []
for K in [5]:  
    for seed in seeds:
        mskf = MultilabelStratifiedKFold(n_splits=K,shuffle=True,random_state=seed)
        name = f"fold_msk_{K}_seed_{seed}"
        df[name] = -1
        for fold, (trn_, val_) in enumerate(mskf.split(df,df[list(LABEL2TYPE)[:-1]])):
            df.loc[val_, name] = fold

In [16]:
df.groupby(name)[list(LABEL2TYPE)[:-1]].sum()

,NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
fold_msk_5_seed_42,,,,,,,
0,178,5,1,7,1,14,0
1,178,5,1,7,1,14,0
2,179,5,1,6,1,15,1
3,178,4,1,6,0,15,1
4,178,5,1,7,1,14,0


In [17]:
model_name = 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/jovyan/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [18]:
# from torch.utils.data import DataLoader

# from model_zoo.models import FeedbackModel,span_nms,aggregate_tokens_to_words

In [19]:
# model = FeedbackModel(model_name,pooling_params={"pooling_name":"MeanPooling",'params':{}})

In [20]:
# df[df.full_text.str.contains('\?')]

In [21]:
# cl = CustomCollator(tokenizer,model)
# dl = DataLoader(ds,collate_fn=cl)

In [22]:
# for data in tqdm(dl):
#     pass

In [23]:
def visualize_data(df):
    
    dmask = df[df.nb_labels>0].reset_index(drop=True)
    ds = FeedbackDataset(dmask,
                     tokenizer,use_re  = False,
                     add_text_prob=1.,
                     replace_text_prob=0.0,
                     attrib_to_replace = ['NAME_STUDENT','EMAIL','USERNAME','ID_NUM',
                                          'PHONE_NUM','URL_PERSONAL','STREET_ADDRESS'],
                     inference=False)
    
    idx = random.choice(ds.df.index)
    dx = ds[idx]
    
    full_text_ds = dx['text']
    tokens_ds = dx['tokens_clean']
    labels_ds = dx['labels']
    offset_map_ds = dx['offset_mapping_init']

    
    full_text = dmask.iloc[idx]['full_text']
    tokens = dmask.iloc[idx]['tokens']
    labels = dmask.iloc[idx]['labels']
    
    
    
    offset_mapping = get_offset_mapping(full_text, tokens)
    offset_mapping_ = [x for (x,y) in zip(offset_mapping,labels) if y!="O"]
    labels_ = [x for x in labels if x!="O"]
    
    print(" -------------------------------------- GT ---------------------------------------")
    print(" --------------------------------------   ---------------------------------------")
    visualize(full_text,offset_mapping_,labels_)
    
    print(" --------------------------------------   ---------------------------------------\n")
    
    print(" -------------------------------------- Edit ---------------------------------------")
    print(" --------------------------------------   ---------------------------------------\n")
#     offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
    offset_mapping_ = [x for (x,y) in zip(offset_map_ds,labels_ds) if y!="O"]
    labels_ = [x for x in labels_ds if x!="O"]
    visualize(full_text_ds,offset_mapping_,labels_)
    
    
#     print(" -------------------------------------- Pred m2---------------------------------------")
#     print(" --------------------------------------   ---------------------------------------\n")
#     offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
#     offset_mapping_ = [x for (x,y,s) in zip(offset_mapping,labels2,scores2) if y!="O" and s>0.15]
#     labels_ = [x for (x,s) in zip(labels2,scores2) if x!="O" and s>0.15]
#     visualize(full_text_ds,offset_mapping_,labels_)
    
    
    
#     print(" -------------------------------------- Pred blend---------------------------------------")
#     print(" --------------------------------------   ---------------------------------------\n")
#     offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
#     offset_mapping_ = [x for (x,y,s) in zip(offset_mapping,labelsbl,scoresbl) if y!="O" and s>0.15]
#     labels_ = [x for (x,s) in zip(labelsbl,scoresbl) if x!="O" and s>0.15]
#     visualize(full_text_ds,offset_mapping_,labels_)

In [52]:
visualize_data(df)

Loaded 945 samples.
using add_text_prob 1.0
 -------------------------------------- GT ---------------------------------------
 --------------------------------------   ---------------------------------------


 --------------------------------------   ---------------------------------------

 -------------------------------------- Edit ---------------------------------------
 --------------------------------------   ---------------------------------------



In [79]:
idx = random.choice(ds.df[ds.df.PHONE_NUM>0].index)
# idx = 428
# doc = 204
# idx = ds.df[ds.df.document==9854].index[0]
# Example usage:
# idx = 80
d = ds[idx]
full_text_ds = d['text']
tokens_ds = d['tokens_clean']
labels_ds = d['labels']
offset_mapping_init = d['offset_mapping_init']
re_offset_mapping = d['re_offset_mapping']
spacy_to_re = d['spacy_to_re']
re_tokens = d['re_tokens']
# offset_mapping_ds = get_offset_mapping(full_text_ds, tokens_ds)
# offset_mapping_ds1 = tokenize_with_spacy(full_text_ds)['offset_mapping']
# offset_mapping_ds = strip_offset_mapping(full_text_ds,offset_mapping_ds)
# idx,ds.df.iloc[idx]['nb_labels']
len(tokens_ds),len(offset_mapping_init),len(re_tokens),len(re_offset_mapping) #len(offset_mapping_ds1),len(tokens_ds)

(695, 695, 695, 695)

In [80]:
idx

350

In [81]:
# ======================================================================================== #
def get_text_start_end(txt, s, search_from=0):
    txt = txt[int(search_from):]
    try:
        idx = txt.find(s)
        if idx >= 0:
            st = idx
            ed = st + len(s)
        else:
            raise ValueError('Error')
    except:
        res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
        if len(res):
            st, ed = res[0][0], res[0][1]
        else:
            m = SequenceMatcher(None, s, txt).get_opcodes()
            for tag, i1, i2, j1, j2 in m:
                if tag == 'replace':
                    s = s[:i1] + txt[j1:j2] + s[i2:]
                if tag == "delete":
                    s = s[:i1] + s[i2:]

            res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
            if len(res):
                st, ed = res[0][0], res[0][1]
            else:
                idx = txt.find(s)
                if idx >= 0:
                    st = idx
                    ed = st + len(s)
                else:
                    st, ed = 0, 0
    return st + search_from, ed + search_from
    
def get_offset_mapping(full_text, tokens):
    offset_mapping = []

    current_offset = 0
    for token in tokens:
        start, end = get_text_start_end(full_text, token, search_from=current_offset)
        offset_mapping.append((start, end))
        current_offset = end

    return offset_mapping

In [82]:
# ds.df.iloc[idx]['document']

In [83]:
full_text = df.iloc[idx]['full_text']
tokens = df.iloc[idx]['tokens']
labels = df.iloc[idx]['labels']
offset_mapping = get_offset_mapping(full_text, tokens)
# idx,df.iloc[idx]['nb_labels']

In [84]:
offset_mapping = get_offset_mapping(full_text, tokens)
for token, offset,l in zip(tokens, offset_mapping,labels):
    if l!="O":
        print(f"{token}: {offset} : {l}")

Jana: (6, 10) : B-NAME_STUDENT
Telfah: (11, 17) : I-NAME_STUDENT
nbarker@hotmail.com: (26, 45) : B-EMAIL
(: (55, 56) : B-PHONE_NUM
820)913: (56, 63) : I-PHONE_NUM
-: (63, 64) : I-PHONE_NUM
3241x894: (64, 72) : I-PHONE_NUM
Jana: (2365, 2369) : B-NAME_STUDENT
Telfah: (2370, 2376) : I-NAME_STUDENT
nbarker@hotmail.com: (2385, 2404) : B-EMAIL
(: (2414, 2415) : B-PHONE_NUM
820)913: (2415, 2422) : I-PHONE_NUM
-: (2422, 2423) : I-PHONE_NUM
3241x894: (2423, 2431) : I-PHONE_NUM
Jana: (2486, 2490) : B-NAME_STUDENT
Telfah: (2491, 2497) : I-NAME_STUDENT
nbarker@hotmail.com: (2506, 2525) : B-EMAIL
(: (2535, 2536) : B-PHONE_NUM
820)913: (2536, 2543) : I-PHONE_NUM
-: (2543, 2544) : I-PHONE_NUM
3241x894: (2544, 2552) : I-PHONE_NUM


In [85]:
d['word_boxes'].shape

torch.Size([695, 4])

In [86]:
d['gt_spans'][-30:,:]

tensor([[665,   7,   1],
        [666,   7,   1],
        [667,   7,   1],
        [668,   7,   1],
        [669,   7,   1],
        [670,   7,   1],
        [671,   7,   1],
        [672,   7,   1],
        [673,   7,   1],
        [674,   7,   1],
        [675,   7,   1],
        [676,   7,   1],
        [677,   7,   1],
        [678,   7,   1],
        [679,   7,   1],
        [680,   7,   1],
        [681,   7,   1],
        [682,   7,   1],
        [683,   7,   1],
        [684,   7,   1],
        [685,   7,   1],
        [686,   7,   1],
        [687,   7,   1],
        [688,   7,   1],
        [689,   7,   1],
        [690,   7,   1],
        [691,   7,   1],
        [692,   7,   1],
        [693,   7,   1],
        [694,   7,   1]])

In [87]:
# text = "Reflection – Visualization   Deiby"
print((full_text_ds))

Name: Henry Email: william32@yahoo.com Mobile: (871)873-8640x14004 | Design Thinking For Innovation | Challenge & Selection Jordan’s economic growth and stability is strengthened by thriving entrepreneurial ecosystem focused on tech innovation and tech-based startups (TBS). While past efforts have improved ecosystem dynamics and innovation input; nonetheless, innovation outputs and socioeconomic impact remains marginal for many reasons, among which are inefficiencies, replication, lack of creativity, and low- quality pipeline. | Mid last year, UVA (University of Bedfordshire) in Jordan decided to contribute to this conversation and to the Jordanian tech ecosystem by establishing, a tech accelerator called (The Belvedere Vodka). I was chosen as the Program Manager for this project, and my current challenge was to develop new models that would steer the university and other educational institutions towards more responsible and value- focused models of entrepreneurial support. | We starte

In [88]:
x = pd.DataFrame({'tokens':tokens_ds,'tokens_re':re_tokens,
              'labels':labels_ds,'spacy_to_re':spacy_to_re})

x[x.spacy_to_re<=8]

,tokens,tokens_re,labels,spacy_to_re
0,Name,Name,O,0
1,:,:,O,1
2,Henry,Henry,NAME_STUDENT,2
3,Email,Email,O,3
4,:,:,O,4
5,william32@yahoo.com,william32@yahoo.com,EMAIL,5
6,Mobile,Mobile,O,6
7,:,:,O,7
8,(,(,PHONE_NUM,8


In [89]:
s = pd.DataFrame({'spacy_to_re':d["spacy_to_re_unique"]})
s.shape

(695, 1)

In [90]:
x.shape

(695, 4)

In [91]:
s = s.merge(x,how='left',on='spacy_to_re')
s[s.tokens!=s.tokens_re]

,spacy_to_re,tokens,tokens_re,labels


In [92]:
offset_mapping = get_offset_mapping(full_text, tokens)
offset_mapping_ = [x for (x,y) in zip(offset_mapping,labels) if y!="O"]
labels_ = [x for x in labels if x!="O"]

In [93]:
visualize(full_text,offset_mapping_,labels_)

In [94]:
# offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
offset_mapping_ = [x for (x,y) in zip(offset_mapping_init,labels_ds) if y!="O"]
labels_ = [x for x in labels_ds if x!="O"]

In [95]:
# for (x,y) in zip(offset,labels_ds):
#     print(y)

In [96]:
visualize(full_text_ds,offset_mapping_,labels_)

In [43]:
from spacy.lang.en import English
en_tokenizer = English().tokenizer
import re
try:
    from faker import Faker
    fake = Faker(locale = ["fr_FR","fr_CA","en_US",'en_UK','de_DE','en_GB','en_IN','it_IT','fr_BE'])
#     Faker.seed(0)
except:
    print('No faker installed')
    
try:
    from spacy.lang.en import English
    EN_TOK = English().tokenizer
except:
    print("No spacy")
    

def tokenize_with_spacy(text, tokenizer=en_tokenizer):
    tokenized_text = tokenizer(text)
    tokens = [token.text for token in tokenized_text]
    offset_mapping = [(token.idx,token.idx+len(token)) for token in tokenized_text]
    return {'tokens': tokens, 'offset_mapping': offset_mapping}


def remove_double_spaces(text):
    # Use a regular expression to replace consecutive spaces with a single space
    # cleaned_text = re.sub(r'\s{2,}', ' | ', text)
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

def clean_text(text):
    text = text.replace(u'\x9d', u' ')
    # text = resolve_encodings_and_normalize(text)
    # text = text.replace(u'\xa0', u' ')
    # text = text.replace(u'\x85', u'\n')
    text = remove_double_spaces(text)
    text = text.strip()
    return text

# ======================================================================================== #
def find_successive_numbers(input_array):
    result = []
    current_sublist = []

    for num in input_array:
        if not current_sublist or num == current_sublist[-1] + 1:
            current_sublist.append(num)
        else:
            result.append(current_sublist)
            current_sublist = [num]

    if current_sublist:
        result.append(current_sublist)

    return result
# ======================================================================================== #

def generate_random_number(length):
    return ''.join(random.choice('0123456789') for _ in range(length))
# ======================================================================================== #
def generate_fake_social_media_urls(num_urls=1):
    social_media_platforms = {
        'LinkedIn': 'linkedin.com/in/',
        'YouTube': 'youtube.com/c/',
        'Instagram': 'instagram.com/',
        'GitHub': 'github.com/',
        'Facebook': 'facebook.com/',
        'Twitter': 'twitter.com/'
    }

    fake_social_media_urls = []

    for _ in range(num_urls):
        fake_user_name = fake.user_name()
        platform, domain = random.choice(list(social_media_platforms.items()))
        fake_url = f'https://{domain}{fake_user_name}'
        fake_social_media_urls.append(fake_url)

    return fake_social_media_urls[0]
    
# ======================================================================================== #
def generate_random_data_with_probabilities():

    name = random.choices([fake.name(),fake.first_name(), fake.last_name()],
                          weights = [0.7,0.15,0.15], k = 1)[0]  #generic.person.full_name()
    phone_number =  fake.phone_number()
    username = fake.user_name()
    email = fake.ascii_free_email()
    address = fake.address()
    id_num = random.choices([fake.passport_number(),fake.bban(), fake.iban(),
                             fake.credit_card_number(),fake.swift(),fake.ean(),
                             generate_random_number(12)],k=1,weights = [1/7]*7)[0]
    url_pers = generate_fake_social_media_urls()

    ret = dict(
              NAME_STUDENT=name,
              EMAIL=email,
              USERNAME=username,
              ID_NUM=id_num,
              URL_PERSONAL=url_pers,
              PHONE_NUM=phone_number,
              STREET_ADDRESS=address
              )

    for k,v in ret.items():
        ret[k] = clean_text(v.replace("\n\n"," | ").replace("\n"," [BR] "))
    return ret
# ======================================================================================== #
def name_student(v):
    # Add Name for Name/ and Mobile/Tel for phone Email for mail
    text = random.choices([f"Reflection – Visualization {v}",f'Person {v}',
                   f"STORYTELLER {v}",f"STORY TELLING {v}",f"{v}"],k=1,weights = [0.125,0.125,0.125,0.125,0.5])[0]
    
    return text
# ======================================================================================== #
def generate_fake_data():
    
    data = generate_random_data_with_probabilities()
    
    NB_PII_MAX = random.choice([1,2,3])
    piis_ent = random.sample(list(data.keys()),k=NB_PII_MAX)
    
    print(piis_ent)
    
    full_text = ""
    tokens = []
    labels = []
    offset_mapping = []
    off = 0
    for num, ent in enumerate(piis_ent):
#         print(ent)
        if ent=="NAME_STUDENT":
            text = name_student(data[ent])
            if num ==0:
                full_text = text
            else:
                off = off + 1 
                full_text = full_text + " " + text
                
            toks = tokenize_with_spacy(text)
            tokns = toks['tokens']
            offset = toks['offset_mapping']
            labs = np.array(["O"]*len(tokns),dtype='<U50')
            idx = [i for i,x in enumerate(tokns) if x in data[ent]][0]
            labs[idx:] = "NAME_STUDENT"
            
            tokens = tokens + tokns
            labels = labels + labs.tolist()
            new_offset = [(x[0]+off,x[1]+off) for x in offset]
            offset_mapping = offset_mapping + new_offset
            off = offset_mapping[-1][1]
#             print(off)
        else:
            text = data[ent]
            
            if num ==0:
                full_text = text
            else:
                off = off+1
                full_text = full_text + " " + text
            
            toks = tokenize_with_spacy(text)
            tokns = toks['tokens']
            offset = toks['offset_mapping']
            labs = [ent]*len(tokns)
            
            tokens = tokens + tokns
            labels = labels + labs
            
            new_offset = [(x[0]+off,x[1]+off) for x in offset]
            offset_mapping = offset_mapping + new_offset
            off = offset_mapping[-1][1]
#             print(off)
#         print(labels)
    return full_text,tokens,labels,offset_mapping

In [100]:
fake.name()

'Tatiana Pezzali-Battisti'

In [741]:
fake.passport_number(),fake.bban(), fake.iban(), fake.credit_card_number(),fake.swift(),fake.ean()

('G92491893',
 '959245225788873344',
 'GB40MWTL93332316312216',
 '676333923347',
 'NUAGFROZFI3',
 '6003834602338')

In [ ]:
fake.license_plate(),fake.ssn(),

In [567]:
fake.password(length=12)

'b_bhu$(x5S9Z'

In [763]:
data = generate_random_data_with_probabilities()
data

{'NAME_STUDENT': 'Vaillant',
 'EMAIL': 'apostolostrommler@yahoo.de',
 'USERNAME': 'massartmatteo',
 'ID_NUM': 'ZKCY5351271851823',
 'URL_PERSONAL': 'https://youtube.com/c/dianacross',
 'PHONE_NUM': '028 9018901',
 'STREET_ADDRESS': 'Buchholzstr. 4 [BR] 37480 Erkelenz'}

In [764]:
a = name_student(data['NAME_STUDENT'])
a

'Vaillant'

In [765]:
tokenize_with_spacy(a)

{'tokens': ['Vaillant'], 'offset_mapping': [(0, 8)]}

In [766]:
NB_PII_MAX = random.choice([1,2,3])
random.sample(list(data.keys()),k=NB_PII_MAX,)

['NAME_STUDENT', 'URL_PERSONAL']

In [767]:
full_text,tokens,labels,offset_mapping = generate_fake_data()

['PHONE_NUM', 'STREET_ADDRESS']


In [768]:
# offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
offset_mapping_ = [x for (x,y) in zip(offset_mapping,labels) if y!="O"]
labels_ = [x for x in labels if x!="O"]
visualize(full_text,offset_mapping_,labels_)

In [769]:
full_text

'+919789633759 728 Murphy square [BR] Harrietville [BR] L31 8QP'

In [770]:
offset_mapping

[(0, 13),
 (14, 17),
 (18, 24),
 (25, 31),
 (32, 33),
 (33, 35),
 (35, 36),
 (37, 49),
 (50, 51),
 (51, 53),
 (53, 54),
 (55, 58),
 (59, 62)]

In [748]:
labels

['PHONE_NUM', 'USERNAME', 'EMAIL']

In [749]:
fake.phone_number()

'+49(0) 067910765'

In [16]:
# import re
# import torch
# import random
# import numpy as np
# import pandas as pd
# from torch.utils.data import Dataset

# from tqdm.auto import tqdm
# from difflib import SequenceMatcher

# import codecs
# import os
# from collections import Counter
# from typing import Dict, List, Tuple

# from text_unidecode import unidecode
# import joblib
# import torch


# try:
#     from faker import Faker
#     fake = Faker()
#     Faker.seed(0)
# except:
#     print('No faker installed')
    
# try:
#     from spacy.lang.en import English
#     EN_TOK = English().tokenizer
# except:
#     print("No spacy")

    
    
# def process_regex(pattern, reverse=False):
#     replacements = {
#         '(': r'\(',
#         ')': r'\)',
#         '[': r'\[',
#         ']': r'\]',
#         '|': r'\|',
#         '?': r'\?',
#         '*': r'\*',
#         '+': r'\+'
#     }
    
#     if reverse:
#         replacements = {v: k for k, v in replacements.items()}
    
#     for old, new in replacements.items():
#         pattern = pattern.replace(old, new)
    
#     return pattern



# LABEL2TYPE = ('NAME_STUDENT','EMAIL','USERNAME','ID_NUM', 'PHONE_NUM','URL_PERSONAL','STREET_ADDRESS','O')
# TYPE2LABEL = {t: l for l, t in enumerate(LABEL2TYPE)}
# ID_TYPE = {"0-0":0,"0-1":1,
#            "1-0":2,"1-1":3,
#            "2-0":4,"2-1":5,
#            "3-0":6,"3-1":7,
#            "4-0":8,"4-1":9,
#            "5-0":10,"5-1":11,
#            "6-0":12,"6-1":13
#           }

# ID_NAME = {"0-0":"B-NAME_STUDENT","0-1":"I-NAME_STUDENT",
#            "1-0":"B-EMAIL","1-1":"I-EMAIL",
#            "2-0":"B-USERNAME","2-1":"I-USERNAME",
#            "3-0":"B-ID_NUM","3-1":"I-ID_NUM",
#            "4-0":"B-PHONE_NUM","4-1":"I-PHONE_NUM",
#            "5-0":"B-URL_PERSONAL","5-1":"I-URL_PERSONAL",
#            "6-0":"B-STREET_ADDRESS","6-1":"I-STREET_ADDRESS",
#            "7-0":"O","7-1":"O"
#           }

# RE_ID_PHONE = r"""(\(?\+\s*\d{1,4}\s*\)?\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\s{0,2}\d{0,5}|\(?\+\s*\d{1,4}\s*\)?\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s{0,2}\d{0,5}|\(?\s*\d{1,4}\s*\)?\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s{0,2}\d{0,5}\s*[\.\-x]?\d{1,5}\s{0,2}\d{0,5}|\(?\s*\d{1,4}\s*\)?\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s*[\.\-x]?\d{1,5}\s{0,2}\d{0,5}|\(\s*\d{3}\s*\)\s*\d{3}\s*\-\s*\d{4}\s*\w{0,3}(\s*\d{1,8}\s*)?|\b\d{2,}-\d{2,}\.\d{2,}\.\d{2,}\.\d{2,}\b|\b\d{2,}-\d{2,}\-\d{2,}\-\d{2,}\-\d{2,}\b|\b\d{2,}\-\d{2,}\-\d{2,}\-\d{2,}\b|\b\d{2,}\.\d{2,}\.\d{2,}\.\d{2,}\b|\d{3}\s*\.\s*\d{3}\s*\.\s*\d{1,5}|\d{3}\s*\-\s*\d{3}\s*\-\s*\d{1,5}|\d{3}\s*x\s*\d{3}\s*x\s*\d{1,5}|\d{1,3}\s{0,2}\d{1,}\s{0,2}\d{1,}|\b\d{1,}\s*\d{1,}\s*\d{1,}|\b\d{2,}\-\d{2,}\-\d{2,}\b|\b\d{2,}\.\d{2,}\.\d{2,}\b|\b\d{1,}-\d{1,}|[\w\.\:\-\_\|]*\d{6,})"""
# REGEX_COMPILE = re.compile(RE_ID_PHONE)


# ## =============================================================================== ##
# class TrainDataset(Dataset):
#     def __init__(self,
#                  df,
#                  tokenizer,
#                  add_text_prob=0.9,
#                  replace_text_prob=0.5,
#                  attrib_to_replace = ['NAME_STUDENT','EMAIL','USERNAME','ID_NUM',
#                                       'PHONE_NUM','URL_PERSONAL','STREET_ADDRESS']
#                  ):
        
#         self.tokenizer = tokenizer
#         self.df = df
#         self.attrib_to_replace = attrib_to_replace

#         print(f'Loaded {len(self)} samples.')

#         assert 0 <= add_text_prob <= 1
#         assert 0 <= replace_text_prob <= 1
#         self.add_text_prob = add_text_prob
#         self.replace_text_prob = replace_text_prob
#     # ======================================================================================== #
#     def __len__(self):
#         return len(self.df)
#     # ======================================================================================== #
#     def __getitem__(self, index):
        
#         df = self.df.iloc[index]
#         text_id = df['document']
        
#         ## Adding space tokens
#         if len(self.tokenizer.encode("\n\n"))==2:
#             text = self.clean_text(df['full_text'].replace("\n\n"," | ").replace("\n"," [BR] "))
#             txt_tokens = [self.clean_text(x.replace("\n\n"," | ").replace("\n"," [BR] ")) for x in df['tokens']]
#         else:
#             text = self.clean_text(df['full_text'])
#             txt_tokens = [self.clean_text(x) for x in df['tokens']]
            
#         labels = df['labels']
#         offset_mapping_init = self.get_offset_mapping(text,txt_tokens)
        
#         ##############   Augmentation   #############
#         # Replace PII
# #         prob = 
#         if np.random.random()< self.replace_text_prob:
#             print("rep")
#             text,labels,txt_tokens = self.create_mapper_n_clean(text,labels,offset_mapping_init,
#                                                                        attribut=self.attrib_to_replace)
#             offset_mapping_init = self.get_offset_mapping(text, txt_tokens)
        

    
#         # Add PII
#         if np.random.random() < self.add_text_prob:
#             print("add")
#             new_text,new_tokens,new_labels,new_offset_mapping = self.generate_fake_data()
            
        
            
#             text,txt_tokens,labels,offset_mapping_init  = self.add_text(text,txt_tokens,labels,offset_mapping_init,
#                                                                    new_text,new_tokens,new_labels,new_offset_mapping)
        
#         ### Convert to regex space
#         re_offset_mapping,spacy_to_re,re_tokens,spacy_to_re_unique = self.spacy_to_re_off(text,txt_tokens,offset_mapping_init)


#         hf_tokens = self.tokenizer(text, return_offsets_mapping=True)
#         input_ids = torch.LongTensor(hf_tokens['input_ids'])
#         attention_mask = torch.LongTensor(hf_tokens['attention_mask'])
#         hf_offset_mapping = np.array(hf_tokens['offset_mapping'])

#         num_tokens = len(input_ids)

#         # token slices of words
#         woff = np.array(re_offset_mapping)
#         toff = np.array(hf_offset_mapping)
#         wx1, wx2 = woff.T
#         tx1, tx2 = toff.T
#         ix1 = np.maximum(wx1[..., None], tx1[None, ...])
#         ix2 = np.minimum(wx2[..., None], tx2[None, ...])
#         ux1 = np.minimum(wx1[..., None], tx1[None, ...])
#         ux2 = np.maximum(wx2[..., None], tx2[None, ...])
#         ious = (ix2 - ix1).clip(min=0) / (ux2 - ux1)
# #         assert (ious > 0).any(-1).all()

#         word_boxes = []
# #         err = []
#         for i,row in enumerate(ious):
#             inds = row.nonzero()[0]
#             try:
#                 word_boxes.append([inds[0], 0, inds[-1] + 1, 1])
#             except:
#                 word_boxes.append([-100, 0, -99, 1])
# #                 err.append(i)
                
#         word_boxes = torch.FloatTensor(word_boxes)


#         gt_spans = []        
#         for i,label in enumerate(labels) :
#             gt_spans.append([i,TYPE2LABEL[label.split('-')[-1] if label!="O" else "O"],0 if label.split('-')[0]=="B" else 1])
            
#         gt_spans = torch.LongTensor(gt_spans)

#         # random mask augmentation
# #         if np.random.random() < self.mask_prob:
# #             all_inds = np.arange(1, len(input_ids) - 1)
# #             n_mask = max(int(len(all_inds) * self.mask_ratio), 1)
# #             np.random.shuffle(all_inds)
# #             mask_inds = all_inds[:n_mask]
# #             input_ids[mask_inds] = self.tokenizer.mask_token_id

#         return dict(
#                     text_id=text_id,
#                     text=text,
#                     labels = labels,
#                     re_tokens = re_tokens,
#                     spacy_to_re = spacy_to_re,
#                     spacy_to_re_unique = spacy_to_re_unique,
#                     tokens = df['tokens'],
#                     tokens_clean = txt_tokens,
#                     input_ids=input_ids,
#                     offset_mapping_init = offset_mapping_init,
#                     re_offset_mapping = re_offset_mapping,
#                     attention_mask=attention_mask,
#                     word_boxes=word_boxes,
#                     gt_spans=gt_spans)
#     # ======================================================================================== #
#     def name_student(self,v):
#         # Add Name for Name/ and Mobile/Tel for phone Email for mail
#         text = random.choices([f"Reflection – Visualization {v}",f'Person {v}',
#                        f"STORYTELLER {v}",f"STORY TELLING {v}",f"{v}"],k=1,weights = [0.125,0.125,0.125,0.125,0.5])[0]

#         return text
#     # ======================================================================================== #
#     def generate_fake_data(self):

#         data = self.generate_random_data_with_probabilities()

#         NB_PII_MAX = random.choice([1,2,3])
#         piis_ent = random.sample(list(data.keys()),k=NB_PII_MAX)


#         full_text = ""
#         tokens = []
#         labels = []
#         offset_mapping = []
#         off = 0
#         for num, ent in enumerate(piis_ent):

#             if ent=="NAME_STUDENT":
#                 text = self.name_student(data[ent])
#                 if num ==0:
#                     full_text = text
#                 else:
#                     off = off + 1 
#                     full_text = full_text + " " + text

#                 toks = self.tokenize_with_spacy(text)
#                 tokns = toks['tokens']
#                 offset = toks['offset_mapping']
#                 labs = np.array(["O"]*len(tokns),dtype='<U50')
#                 idx = [i for i,x in enumerate(tokns) if x in data[ent]][0]
#                 labs[idx:] = "NAME_STUDENT"

#                 tokens = tokens + tokns
#                 labels = labels + labs.tolist()
#                 new_offset = [(x[0]+off,x[1]+off) for x in offset]
#                 offset_mapping = offset_mapping + new_offset
#                 off = offset_mapping[-1][1]

#             else:
#                 text = data[ent]

#                 if num ==0:
#                     full_text = text
#                 else:
#                     off = off+1
#                     full_text = full_text + " " + text

#                 toks = self.tokenize_with_spacy(text)
#                 tokns = toks['tokens']
#                 offset = toks['offset_mapping']
#                 labs = [ent]*len(tokns)

#                 tokens = tokens + tokns
#                 labels = labels + labs

#                 new_offset = [(x[0]+off,x[1]+off) for x in offset]
#                 offset_mapping = offset_mapping + new_offset
#                 off = offset_mapping[-1][1]

#         return full_text,tokens,labels,offset_mapping

#     def remove_double_spaces(self,text):
#         # Use a regular expression to replace consecutive spaces with a single space
#         # cleaned_text = re.sub(r'\s{2,}', ' | ', text)
#         cleaned_text = re.sub(r'\s+', ' ', text)
#         return cleaned_text

#     def clean_text(self,text):
#         text = text.replace(u'\x9d', u' ')
#         # text = resolve_encodings_and_normalize(text)
#         # text = text.replace(u'\xa0', u' ')
#         # text = text.replace(u'\x85', u'\n')
#         text = self.remove_double_spaces(text)
#         text = text.strip()
#         return text

#     # ======================================================================================== #
#     def find_successive_numbers(self,input_array):
#         result = []
#         current_sublist = []

#         for num in input_array:
#             if not current_sublist or num == current_sublist[-1] + 1:
#                 current_sublist.append(num)
#             else:
#                 result.append(current_sublist)
#                 current_sublist = [num]

#         if current_sublist:
#             result.append(current_sublist)

#         return result
#     # ======================================================================================== #

#     def generate_random_number(self,length):
#         return ''.join(random.choice('0123456789') for _ in range(length))
#     # ======================================================================================== #
#     def generate_fake_social_media_urls(self,num_urls=1):
#         social_media_platforms = {
#             'LinkedIn': 'linkedin.com/in/',
#             'YouTube': 'youtube.com/c/',
#             'Instagram': 'instagram.com/',
#             'GitHub': 'github.com/',
#             'Facebook': 'facebook.com/',
#             'Twitter': 'twitter.com/'
#         }

#         fake_social_media_urls = []

#         for _ in range(num_urls):
#             fake_user_name = fake.user_name()
#             platform, domain = random.choice(list(social_media_platforms.items()))
#             fake_url = f'https://{domain}{fake_user_name}'
#             fake_social_media_urls.append(fake_url)

#         return fake_social_media_urls[0]
#     # ======================================================================================== #
#     def generate_random_data_with_probabilities(self):

#         name = random.choices([fake.name(),fake.first_name(), fake.last_name()],
#                               weights = [0.7,0.15,0.15], k = 1)[0]  #generic.person.full_name()
#         phone_number =  fake.phone_number()
#         username = fake.user_name()
#         email = fake.ascii_free_email()
#         address = fake.address()
#         id_num = random.choices([fake.passport_number(),fake.bban(),
#                                  fake.iban(),self.generate_random_number(12)],k=1,weights = [0.1,0.10,0.15,0.65])[0]
#         url_pers = self.generate_fake_social_media_urls()

#         ret = dict(
#                   NAME_STUDENT=name,
#                   EMAIL=email,
#                   USERNAME=username,
#                   ID_NUM=id_num,
#                   URL_PERSONAL=url_pers,
#                   PHONE_NUM=phone_number,
#                   STREET_ADDRESS=address
#                   )

#         for k,v in ret.items():
#             ret[k] = self.clean_text(v.replace("\n\n"," | ").replace("\n"," [BR] "))
#         return ret
#     # ======================================================================================== #
#     def generate_ent(self,text,labels,offset_mapping):

#         idx_lab = np.argwhere(np.array(labels)!="O").reshape(-1)
#         pos = self.find_successive_numbers(idx_lab)
#         lab = np.array(labels)


#         ent = {}
#         ent_order = []
#         ent_offset_in_order = []
#         for i,p in enumerate(pos):
#             l = [x.split('-')[-1] for x in lab[p]]

#             if len(np.unique(l))>1:
#                 px = self.successive_positions(l)
#                 for pp in px:
#                     full_name = text[offset_mapping[p[pp[0]]][0]:offset_mapping[p[pp[-1]]][1]].strip()
#                     ent[full_name] = l[pp[-1]]
#                     ent_order.append(full_name)
#                     ent_offset_in_order.append((offset_mapping[p[pp[0]]][0],
#                                                 offset_mapping[p[pp[-1]]][1]))

#             else:
#                 full_name = text[offset_mapping[p[0]][0]:offset_mapping[p[-1]][1]].strip()
#                 ent[full_name] = l[-1]
#                 ent_order.append(full_name)
#                 ent_offset_in_order.append((offset_mapping[p[0]][0],offset_mapping[p[-1]][1]))

#         return ent,ent_order,ent_offset_in_order
#     # ======================================================================================== #
#     def tokenize_with_spacy(self,text,tok=EN_TOK):
#         tokenized_text = tok(text)
#         tokens = [token.text for token in tokenized_text]
#         offset_mapping = [(token.idx,token.idx+len(token)) for token in tokenized_text]
#         return {'tokens': tokens, 'offset_mapping': offset_mapping}
    
#     def successive_positions(self,input_list):
#         result = []
#         current_group = []
#         prev_element = None

#         for i, element in enumerate(input_list):
#             if element == prev_element:
#                 current_group.append(i)
#             else:
#                 if current_group:
#                     result.append(current_group)
#                 current_group = [i]
#             prev_element = element

#         if current_group:
#             result.append(current_group)

#         return result

#     # ======================================================================================== #
#     def get_offset_mapping(self,full_text, tokens):
#         offset_mapping = []

#         current_offset = 0
#         for token in tokens:
#             start, end = self.get_text_start_end(full_text, token, search_from=current_offset)
#             offset_mapping.append((start, end))
#             current_offset = end

#         return offset_mapping
#     # ======================================================================================== #
#     def create_mapper_n_clean(self,full_text,labels,offset_mapping,attribut=["NAME_STUDENT"]):
#         ent,ent_order,ent_offset_in_order = self.generate_ent(full_text,labels,offset_mapping)

#         mapper = {}
#         label_mapper = {}
#         new_tokens = []
#         txt_added = 0
#         for num,k in enumerate(ent_order):
#             v = ent[k]

#             if v in attribut:      
#                 dc_ent = self.generate_random_data_with_probabilities()
#                 mapper[k] = dc_ent[v]
#                 label_mapper[dc_ent[v]] = v

#                 old_len = len(full_text)
#                 if k in mapper.keys():
#                     full_text = full_text[:ent_offset_in_order[num][0]+txt_added] +" " +mapper[k] + " "+full_text[ent_offset_in_order[num][-1]+txt_added:]
#                     txt_added+= len(full_text)-old_len

#                     new_tokens.append(mapper[k])
#                 else:
#                     full_text = full_text[:ent_offset_in_order[num][0]+txt_added] + " "+dc_ent[v] +" "+ full_text[ent_offset_in_order[num][-1]+txt_added:]

#                     new_tokens.append(dc_ent[v])
#                     txt_added+= len(full_text)-old_len
#             else:
#                 label_mapper[k] = v
#                 new_tokens.append(k)

#         full_text = self.clean_text(full_text)

#         tokenized_text = self.tokenize_with_spacy(full_text)
#         tokens = tokenized_text['tokens']
#         tg = self.get_offset_mapping(full_text, new_tokens)


#         woff = np.array(tokenized_text['offset_mapping'])
#         labels = np.array(["O"]*len(woff),dtype='<U50')

#         toff = np.array(tg)
#         wx1, wx2 = woff.T
#         tx1, tx2 = toff.T
#         ix1 = np.maximum(wx1[..., None], tx1[None, ...])
#         ix2 = np.minimum(wx2[..., None], tx2[None, ...])
#         ux1 = np.minimum(wx1[..., None], tx1[None, ...])
#         ux2 = np.maximum(wx2[..., None], tx2[None, ...])
#         ious = (ix2 - ix1).clip(min=0) / (ux2 - ux1)


#         for i,row in enumerate(ious):
#             inds = row.nonzero()[0]
#             if len(inds):
#                 labels[i] = label_mapper[new_tokens[inds[0]]]

#         labels = labels.tolist()

#         return full_text,labels,tokens

#     # ======================================================================================== #
#     def get_text_start_end(self,txt, s, search_from=0):
#         txt = txt[int(search_from):]
#         try:
#             idx = txt.find(s)
#             if idx >= 0:
#                 st = idx
#                 ed = st + len(s)
#             else:
#                 raise ValueError('Error')
#         except:
#             res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
#             if len(res):
#                 st, ed = res[0][0], res[0][1]
#             else:
#                 m = SequenceMatcher(None, s, txt).get_opcodes()
#                 for tag, i1, i2, j1, j2 in m:
#                     if tag == 'replace':
#                         s = s[:i1] + txt[j1:j2] + s[i2:]
#                     if tag == "delete":
#                         s = s[:i1] + s[i2:]

#                 res = [(m.start(0), m.end(0)) for m in re.finditer(s, txt)]
#                 if len(res):
#                     st, ed = res[0][0], res[0][1]
#                 else:
#                     idx = txt.find(s)
#                     if idx >= 0:
#                         st = idx
#                         ed = st + len(s)
#                     else:
#                         st, ed = 0, 0
#         return st + search_from, ed + search_from
    
#     # ======================================================================================== #
#     def find_patterns(self,text,regex):
#         matches = [(match.group(0), match.start(), match.end()) for match in regex.finditer(text)]
#         offsets = self.strip_offset_mapping(text,[(m[1],m[2]) for m in matches])
#         return [m[0].strip() for m in matches],offsets
#     # ======================================================================================== #
#     def spacy_to_re_off(self,text,tokens,offset_mapping_init):
        

#         lab,off_matc = self.find_patterns(text,REGEX_COMPILE)


#         if len(lab):

#             spacy_to_re = []
#             re_oof = []
#             tokens_re = []

#             # token slices of words
#             woff = np.array(offset_mapping_init)
#             toff = off_matc
#             wx1, wx2 = woff.T
#             tx1, tx2 = toff.T
#             ix1 = np.maximum(wx1[..., None], tx1[None, ...])
#             ix2 = np.minimum(wx2[..., None], tx2[None, ...])
#             ux1 = np.minimum(wx1[..., None], tx1[None, ...])
#             ux2 = np.maximum(wx2[..., None], tx2[None, ...])
#             ious = (ix2 - ix1).clip(min=0) / (ux2 - ux1+1e-12)

#             for i,(spcy_of_set,tok,row) in enumerate(zip(offset_mapping_init,tokens,ious)):
#                 inds = row.nonzero()[0]
#                 if len(inds):
#                     spacy_to_re.append(inds[0])
#                     re_oof.append((off_matc[inds[0]].tolist()[0],off_matc[inds[0]].tolist()[1]))
#                     tokens_re.append(lab[inds[0]] if len(lab[inds[0]])>len(tok) else tok)
#                 else:
#                     spacy_to_re.append(len(lab)+i)
#                     re_oof.append(spcy_of_set)
#                     tokens_re.append(tok)

#             re_oof = [(x,i) for i, x in enumerate(re_oof) if re_oof.index(x) == i]
#             spacy_to_re_unique = [spacy_to_re[x[1]] for x in re_oof]
#             re_oof = [x[0] for x in re_oof]
#         else:
#             spacy_to_re = np.arange(len(offset_mapping_init)).tolist()
#             re_oof = offset_mapping_init
#             tokens_re = tokens

#         return re_oof,spacy_to_re,tokens_re,spacy_to_re_unique
#     # ======================================================================================== #
#     def strip_offset_mapping(self, text, offset_mapping):
#         ret = []
#         for start, end in offset_mapping:
#             match = list(re.finditer('\S+', text[start:end]))
#             if len(match) == 0:
#                 ret.append((start, end))
#             else:
#                 span_start, span_end = match[0].span()
#                 ret.append((start + span_start, start + span_end))
#         return np.array(ret)
#     # ======================================================================================== #
#     def get_word_offsets(self, text):
#         matches = re.finditer("\S+", text)
#         spans = []
#         words = []
#         for match in matches:
#             span = match.span()
#             word = match.group()
#             spans.append(span)
#             words.append(word)
#         assert tuple(words) == tuple(text.split())
#         return np.array(spans)
#     # ======================================================================================== #
#     def custom_distribution(self,n):
#         distribution = [0] * n
#         middle_index = n // 2
#         for i in range(middle_index):
#             distribution[i] = (middle_index - i) / middle_index
#             distribution[n - 1 - i] = (middle_index - i) / middle_index
#         return distribution
#     # ======================================================================================== #
#     def add_text(self,full_text,tokens,labels,offset_mapping,
#                  new_text,new_tokens,new_labels,new_offset_mapping):
#         try:
#             s = full_text.split('|')
#             prob_dist = self.custom_distribution(len(s))
#             id_ = random.choices(np.arange(len(s)),k=1,weights = prob_dist)[0]

#             idx = [len(s[i]) for i in range(id_+1)]
#             idx = sum(idx)
#             full_text = full_text[:idx+id_+1] +" "+ new_text + full_text[idx+id_+1:]


#             t_idx = [i for i,x in enumerate(offset_mapping) if x[1]==idx+id_+1][-1]+1


#             tokens = tokens[:t_idx]+new_tokens+tokens[t_idx:]
#             labels = labels[:t_idx]+new_labels+labels[t_idx:]


#             v = offset_mapping[:t_idx][-1][1]
#             new_offset_mappings = [(x[0]+v+1,x[1]+v+1) for x in new_offset_mapping]
#             v1 = new_offset_mappings[-1][1]
#             vx = v1-v
            
#             old_offset_mapping =  [(x[0]+vx,x[1]+vx) for x in offset_mapping[t_idx:]]
#             offset_mapping = offset_mapping[:t_idx]+new_offset_mappings+old_offset_mapping

#         except:
#             print("Text not added")
#         return full_text,tokens,labels,offset_mapping

# ## =============================================================================== ##
# ##                                                                                 ##
# ## =============================================================================== ##

# class CustomCollator(object):
#     def __init__(self, tokenizer, model):
#         self.pad_token_id = tokenizer.pad_token_id
#         if hasattr(model.config, 'attention_window'):
#             # For longformer
#             # https://github.com/huggingface/transformers/blob/v4.17.0/src/transformers/models/longformer/modeling_longformer.py#L1548
#             self.attention_window = (model.config.attention_window
#                                      if isinstance(
#                                          model.config.attention_window, int)
#                                      else max(model.config.attention_window))
#         else:
#             self.attention_window = None

#     def __call__(self, samples):
#         batch_size = len(samples)
#         assert batch_size == 1, f'Only batch_size=1 supported, got batch_size={batch_size}.'

#         sample = samples[0]

#         max_seq_length = len(sample['input_ids'])
#         if self.attention_window is not None:
#             attention_window = self.attention_window
#             padded_length = (attention_window -
#                              max_seq_length % attention_window
#                              ) % attention_window + max_seq_length
#         else:
#             padded_length = max_seq_length

#         input_shape = (1, padded_length)
#         input_ids = torch.full(input_shape,
#                                self.pad_token_id,
#                                dtype=torch.long)
#         attention_mask = torch.zeros(input_shape, dtype=torch.long)

#         seq_length = len(sample['input_ids'])
#         input_ids[0, :seq_length] = sample['input_ids']
#         attention_mask[0, :seq_length] = sample['attention_mask']

#         text_id = sample['text_id']
#         tokens = sample['tokens']
#         tokens_clean = sample['tokens_clean']
#         re_tokens = sample['re_tokens']
#         spacy_to_re = sample['spacy_to_re']
#         spacy_to_re_unique = sample['spacy_to_re_unique']
#         word_boxes = sample['word_boxes']
#         gt_spans = sample['gt_spans']

#         return dict(text_id=text_id,
#                     tokens_clean=tokens_clean,
#                     tokens = tokens,
#                     re_tokens = re_tokens,
#                     spacy_to_re = spacy_to_re,
#                     spacy_to_re_unique = spacy_to_re_unique,
#                     input_ids=input_ids,
#                     attention_mask=attention_mask,
#                     word_boxes=word_boxes,
#                     gt_spans=gt_spans)